In [1]:
import numpy as np
import zebende as zb
import pandas as pd
from pathlib import Path

In [12]:
# criando um vetor com as jalenas temporais
tws = np.array([4, 9, 28, 60, 104, 407, 4564, 35212, 86015, 252313], dtype=np.int64) 

In [28]:
# criando uma lista de arquivos a serem processados
root = Path()
data_dir = root / 'data_eeg'
file_gen = data_dir.glob('*.txt')
file_list = list(file_gen)
file_list

[WindowsPath('data_eeg/PN00-1_data.txt'),
 WindowsPath('data_eeg/PN00-2_data.txt'),
 WindowsPath('data_eeg/PN00-3_data.txt'),
 WindowsPath('data_eeg/PN00-4_data.txt'),
 WindowsPath('data_eeg/PN00-5_data.txt')]

In [32]:
# mude o indice para mudar o arquivo processado - não precisa mudar o nome do arquivo de saída
file_index = 0
# extraindo o nome do arquivo sem a extensão.
file_stem = file_list[file_index].stem
# carregando o arquivo selecionado
data = pd.read_csv(file_list[file_index])
data.head(), data.tail()

(       Time      1       2      3       4       5       6      7      8  \
 0  0.000000  1.875 -10.000 -0.875 -10.875  -5.625 -26.375  2.250  8.250   
 1  0.001953 -0.375 -12.500  0.500 -12.125 -14.375 -29.500 -3.500 -0.375   
 2  0.003906 -3.250 -12.500  1.750 -14.750 -24.125 -31.000 -5.125 -8.125   
 3  0.005859  0.375 -10.125  4.000 -13.250 -17.500 -28.500 -1.250 -1.750   
 4  0.007812  3.125  -7.250  3.875 -13.500 -18.625 -22.125  1.750 -2.250   
 
        9  ...     26     27      28     29           30   31   32      33  \
 0  0.625  ...  8.375  5.375   3.500  1.500  7686.000385  0.0  0.0 -15.875   
 1  1.375  ...  6.625  7.500  10.000 -4.375  7762.250388  0.0  0.0 -18.000   
 2  2.000  ...  8.500  9.500  15.625 -0.625  7838.500392  0.0  0.0 -25.375   
 3  3.000  ...  7.250  8.375  11.000 -0.750  7899.500395  0.0  0.0 -34.250   
 4  3.000  ...  9.125  5.250   5.125  7.250  7884.250394  0.0  0.0 -43.500   
 
         34    35  
 0 -187.875  40.0  
 1 -209.000  40.0  
 2 -211.125 

In [33]:
# filtrando as series que serão processadas 
data = data.iloc[:, 1:30]
# convertendo em uma matriz numpy
data_np = data.to_numpy()
data_np.shape, data_np

((1344000, 29),
 array([[  1.875, -10.   ,  -0.875, ...,   5.375,   3.5  ,   1.5  ],
        [ -0.375, -12.5  ,   0.5  , ...,   7.5  ,  10.   ,  -4.375],
        [ -3.25 , -12.5  ,   1.75 , ...,   9.5  ,  15.625,  -0.625],
        ...,
        [ 16.25 ,  12.75 ,  11.875, ...,  16.5  ,  24.375,  24.125],
        [ 13.25 ,   9.25 ,   9.75 , ...,  12.875,  21.875,  21.625],
        [  8.875,   9.   ,   7.875, ...,  11.5  ,  20.75 ,  22.375]]))

In [5]:
# calculando a série integrada
int_data = zb.integrated_series(data_np)
int_data

array([[-2.92852334e+00, -1.53181278e+01],
       [-8.10704669e+00, -3.31362556e+01],
       [-1.61605700e+01, -5.09543834e+01],
       ...,
       [-1.25179569e+01, -7.61376308e+00],
       [-4.07148028e+00, -3.68189087e+00],
       [-3.62702056e-06, -1.86563898e-05]])

In [6]:
# calculando o dfa
dfa, dcca, Pdcca = zb.pdcca(int_data, tws)

In [7]:
dfa

array([[4.22944952, 4.38531531],
       [9.13674333, 9.13797854]])

In [ ]:
# Colocando as escalas temporais ao lado dos cálculos do dfa. 
dfa_concat = np.c_[tws, dfa]
dcca_concat = np.c_[tws, dcca]
Pdcca_concat = np.c_[tws, Pdcca]


In [31]:
# gravando os resultados no sistema de arquivos
pd_dfa = pd.DataFrame(dfa_concat)
pd_dcca = pd.DataFrame(dcca_concat)
pd_pdcca = pd.DataFrame(Pdcca_concat)
out_dfa_path = data_dir / 'dfa_out_{}.dat'.format(file_stem)
out_dcca_path = data_dir / 'dcca_out_{}.dat'.format(file_stem)
out_pdcca_path = data_dir / 'pdcca_out_{}.dat'.format(file_stem)
pd_dfa.to_csv('./data_eeg/dfa_out_{}.dat'.format(file_stem), sep=' ')
pd_dcca.to_csv('./data_eeg/dcca_out_{}.dat'.format(file_stem), sep=' ')
pd_pdcca.to_csv('./data_eeg/pdcca_out_{}.dat'.format(file_stem), sep=' ')